In [ ]:
# Generate the constants
N = 100
r = 10 #um
SA = 4*pi*r 
V = (4/3)*pi*r^2
mem_thickness = 0.01
n = (mem_thickness * SA) / V

Ax = Array(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
Ax[1,end] = 1.0
Ax[end,1] = 1.0
dx = (r*sqrt(pi))/N
Ax = Ax/(dx^2) # adjust for 1/microns
Ay = copy(Ax)

r0 = zeros(100,100,9)
r0[:,:,1] .= 32*2 .*(rand.())   # Cdc42-GTPm
r0[:,:,2] .= 22.2*2 .*(rand.())   # Cdc42-GDPm
r0[:,:,3] .= .4              # Cdc42-GDPc
# total GTPase = 1uM
r0[:,:,4] .= 0.017             # BemGEFc
r0[:,:,5] .= 0.0431                # BemGEFm
r0[:,:,6] .= 1.4                # BemGEF42
# total BemGEF = 0.006uM
r0[:,:,7] .= 0              # Pak1m
r0[:,:,8] .= 0.5               # Pak1c
r0[:,:,9] .= 0                  # Pak1p
# total PAK1 = 0.5uM

# Dummy parameters used only locally in fxn but passed to specify scope, or something..
Ayt = zeros(N,N)
Ayd = zeros(N,N)
tAx = zeros(N,N)
dAx = zeros(N,N)
D42t = zeros(N,N)
D42d = zeros(N,N)
DBG = zeros(N,N)
DBG42 = zeros(N,N)
Dpak = zeros(N,N)
Dpakp = zeros(N,N)
dummies = (Ayt, Ayd, tAx, dAx, D42t, D42d, DBG, DBG42, Dpak, Dpakp)

In [ ]:
function mechanistic!(dr,r,p,t)
    #k1a, k1b, k2a, k2b, k3, k4a, k4b, k5a, k7, k8a, k8b, k9, k10, Dm, Dm2, n, Ax, Ay, tAx, Ayt, dummies = p
    #Ayt, Ayd, tAx, dAx, D42t, D42d, DBG, DBG42, Dpak, Dpakp = dummies
    # Window variables
    rhoT = @view r[:,:,1]
    rhoDm = @view r[:,:,2]
    rhoDc = @view r[:,:,3]
    BGc = @view r[:,:,4]
    BGm = @view r[:,:,5]
    BG42 = @view r[:,:,6]
    Pak1m = @view r[:,:,7]
    Pak1c = @view r[:,:,8]
    Pak1p = @view r[:,:,8]
    # Calculate diffusion
    mul!(Ayt,Ay,rhoT)
    mul!(tAx,rhoT,Ax)
    @. D42t = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,rhoDm)
    mul!(tAx,rhoDm,Ax)
    @. D42d = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,BGm)
    mul!(tAx,BGm,Ax)
    @. DBG = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,BG42)
    mul!(tAx,BG42,Ax)
    @. DBG42 = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,Pak1m)
    mul!(tAx,Pak1m,Ax)
    @. Dpak = Dm2*(Ayt + tAx)
    mul!(Ayt,Ay,Pak1p)
    mul!(tAx,Pak1p,Ax)
    @. Dpakp = Dm2*(Ayt + tAx)
    # Calculate reactions, add diffusion
    @. dr[:,:,1] = (k2a*BGm + k3*BG42) * rhoDm - (k2b + k4a*BGm + k7*BGc) * rhoT + k4b*BG42 + k10*Pak1p*BG42 + D42t # rhoT
    @. dr[:,:,2] = k2b*rhoT - (k2a*BGm + k3*BG42)*rhoDm - k5b*rhoDm + k5a*rhoDc + D42d # rhoDm
    dr[:,:,3] .= (k5b*mean(rhoDm) - k5a*mean(rhoDc)) * n # rhoDc
    dr[:,:,4] .= (k1b*mean(BGm) - mean(BGc)*(k1a + k7*mean(rhoT)) + mean(k10*Pak1p*BG42)) * n #BGc
    @. dr[:,:,5] = k1a*BGc - k1b*BGm + k4b*BG42 - k4a*BGm*rhoT + DBG # BGm
    @. dr[:,:,6] = (k4a*BGm + k7*BGc) * rhoT - k4b*BG42 - k10*Pak1p*BG42 + DBG42 # BG42
    @. dr[:,:,7] = k8a*Pak1c*(rhoT+BG42) - k8b*(Pak1m) - k9*Pak1m*Pak1m*rhoT + Dpak # Pak1m
    dr[:,:,8] .= (k8b*(mean(Pak1m)) - k8a*mean(Pak1c)*(mean(rhoT)+mean(BG42)) + k8b*mean(Pak1p)) * n # Pak1c
    @. dr[:,:,9] = k9*Pak1m*Pak1m*rhoT - k8b*(Pak1p) + Dpakp
    
  end